In [1]:
import pandas as pd
import pickle

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-07-15 22:19:51.657238: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-15 22:19:51.660604: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-15 22:19:51.660615: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Data preprocessing 

In [3]:
import text_preprocessing as tp

[nltk_data] Downloading package wordnet to /home/rim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rim/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
def preprocessing_pipeline(df):
    new_df = df

    for column in new_df:
        print(column)
        df[column] = df[column].apply(lambda x: tp.replace_contractions(x))
        df[column] = df[column].apply(lambda x: tp.remove_URL(x))
        df[column] = df[column].apply(lambda x: tp.remove_html_tags(x))
        df[column] = df[column].apply(lambda x: tp.remove_digits(x))
        df[column] = df[column].apply(lambda x: tp.tokenize(x))
        df[column] = df[column].apply(lambda x: tp.remove_punctuation(x))
        df[column] = df[column].apply(lambda x: tp.to_lowercase(x))
        df[column] = df[column].apply(lambda x: tp.remove_stopwords(x))
        df[column] = df[column].apply(lambda x: tp.lemmatize_words(x))
    return new_df          

In [5]:
df = pd.read_csv('test_data.csv')
df

,t,l
0,I am still waiting on my card?,11
1,What can I do if my card still hasn't arrived ...,11
2,I have been waiting over a week. Is the card s...,11
3,Can I track my card while it is in the process...,11
4,"How do I know if I will get my card, or if it ...",11
...,...,...
9998,You provide support in what countries?,24
9999,What countries are you supporting?,24
10000,What countries are getting support?,24
10001,Are cards available in the EU?,24


In [6]:
import string
def remove_punctuations_from_string(text):
    new_string = text.translate(str.maketrans('', '', string.punctuation))
    return new_string


In [7]:
column = 't'
df[column] = df[column].apply(lambda x: tp.remove_punctuations_from_string(x))
df['t'] = preprocessing_pipeline(df[['t']])
df['l'] =  df['l']

df

t


/tmp/ipykernel_11446/3940928008.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: tp.replace_contractions(x))
/tmp/ipykernel_11446/3940928008.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: tp.remove_URL(x))
/tmp/ipykernel_11446/3940928008.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,t,l
0,"[still, wait, card]",11
1,"[card, still, arrive, week]",11
2,"[wait, week, card, still, come]",11
3,"[track, card, process, delivery]",11
4,"[know, get, card, lose]",11
...,...,...
9998,"[provide, support, country]",24
9999,"[country, support]",24
10000,"[country, get, support]",24
10001,"[card, available, eu]",24


In [8]:
df['t']

0                     [still, wait, card]
1             [card, still, arrive, week]
2         [wait, week, card, still, come]
3        [track, card, process, delivery]
4                 [know, get, card, lose]
                       ...               
9998          [provide, support, country]
9999                   [country, support]
10000             [country, get, support]
10001               [card, available, eu]
10002                [country, represent]
Name: t, Length: 10003, dtype: object

In [8]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

def untokenize(sents):
    return TreebankWordDetokenizer().detokenize(sents)

In [9]:
sents =  df['t'].apply(lambda x: untokenize(x)).tolist()

In [10]:
sents[:5]

['still wait card',
 'card still arrive week',
 'wait week card still come',
 'track card process delivery',
 'know get card lose']

## Bag of Words

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
# cv = CountVectorizer(ngram_range=(2,2))

X = cv.fit_transform(sents).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
X.shape

(10003, 1635)

In [13]:
import pickle

pickle.dump( X, open( "X_BOW.p", "wb" ) )

In [14]:
# sorted(cv.vocabulary_.keys())

In [15]:
Y = pd.get_dummies(df['l']).values
print(Y.shape)

(10003, 77)


In [16]:
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [17]:
pickle.dump( Y, open( "Y.p", "wb" ) )

## TF-IDF 

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
transformed = tfidf.fit_transform(sents)

X_IDF = pd.DataFrame(transformed[0].T.todense(),
    	index=tfidf.get_feature_names(), columns=["TF-IDF"])
X_IDF = X_IDF.sort_values('TF-IDF', ascending=False)
X_IDF

/home/rim/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,TF-IDF
wait,0.734018
still,0.603407
card,0.311638
able,0.000000
quickly,0.000000
...,...
fin,0.000000
fill,0.000000
file,0.000000
figure,0.000000


## 3. word2vec

In [19]:
from gensim import models


In [20]:
w2v = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [21]:
w2v

In [22]:
w2v['healthy']

array([-0.14550781,  0.24804688, -0.15136719,  0.24609375, -0.14746094,
        0.04296875,  0.03515625, -0.01019287,  0.06103516, -0.12597656,
       -0.16796875, -0.078125  ,  0.12792969,  0.21777344, -0.08447266,
        0.01904297,  0.1484375 ,  0.13085938, -0.14941406,  0.12451172,
        0.18847656,  0.09082031, -0.18359375,  0.14453125, -0.10009766,
       -0.07714844, -0.3515625 ,  0.09082031,  0.00494385, -0.02148438,
       -0.02270508, -0.07080078,  0.19824219, -0.14648438, -0.11376953,
        0.08300781, -0.03857422, -0.15136719, -0.15429688,  0.25195312,
        0.12988281, -0.2734375 , -0.14550781,  0.03613281, -0.02441406,
       -0.05004883,  0.09667969,  0.05786133,  0.13085938,  0.04296875,
       -0.06079102, -0.06347656, -0.20703125,  0.08740234,  0.03466797,
       -0.06982422,  0.05371094, -0.31445312, -0.12988281, -0.20507812,
       -0.08886719,  0.13476562, -0.33984375,  0.09277344,  0.16210938,
        0.03271484, -0.05981445,  0.05297852, -0.03295898,  0.03

In [23]:
vect = w2v['healthy']
vect.shape
# w2v.most_similar('happy')

(300,)

In [24]:
# train our custom model   https://radimrehurek.com/gensim/models/word2vec.html

# sents = ['coronavirus is a highly infectious disease',
#    'coronavirus affects older people the most', 
#    'older people are at high risk due to this disease']

sents = [sent.split() for sent in sents]
custom_model = models.Word2Vec(sents, min_count=1,vector_size=1000,workers=4)
custom_model.save("word2vec.model")

custom_model = models.Word2Vec.load("word2vec.model")

vect = custom_model.wv['people']
vect.shape

(1000,)